# Paperspace Stable Diffusion Notebook

This Notebook contains instructions and easy steps for generating images with Stable Diffusion on any of Paperspace's GPUs with little set up required. Follow the instructions in the markdown cells in this Notebook to quickly launch the Stable Diffusion Web UI or begin generating images using the Diffusers Stable Diffusion Pipeline.

Contents:

- Use the Stable Diffusion Web UI
    - Loading in the Stable Diffusion models for the Web UI with support for v1-5 and v2 models
- The Diffusers Stable Diffusion Pipeline
    - Loading in the Stable Diffusion models for the Diffusers StableDiffusionPipeline with support for v1-4, v1-5 and v2 models
    - Access additional models: log in to HuggingFace for online access model files
    - Inference with the Diffusers pipeline
- CLIP reranking
- GFPGAN face restoration and image upscaling
- CodeFormer face restoration and image upscaling

## Setup

First, run the cell below to get everything we need for this Notebook to run.

In [ ]:
!pip install --upgrade git+https://github.com/huggingface/diffusers.git 
!pip install -U transformers accelerate scipy
!pip install flax==0.5.0 --no-deps
!pip install msgpack rich optax accelerate ftfy scipy
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
!mkdir outputs
!pip uninstall -y torch torchvision torchaudio
!pip3 install torch torchvision torchaudio --no-cache-dir --index-url https://download.pytorch.org/whl/cu118
%cd /notebooks/stable-diffusion-webui/extensions
!rm -rf ./stable-diffusion-webui-nsfw-censor
!rm -rf ./sd-webui-controlnet
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-nsfw-censor
!git clone https://github.com/Mikubill/sd-webui-controlnet
%cd /notebooks/stable-diffusion-webui/extensions/sd-webui-controlnet/models
!mkdir /datasets/lora/* /notebooks/stable-diffusion-webui/models/Lora
!cp /datasets/lora/* /notebooks/stable-diffusion-webui/models/Lora
!pip install httpx==0.24.1
%cd /notebooks

In [ ]:
!wget https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -O /notebooks/stable-diffusion-webui/models/Stable-diffusion/sd_xl_base_1.0.safetensors!wget https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -O /notebooks/stable-diffusion-webui/models/Stable-diffusion/sd_xl_refiner_1.0.safetensors!wget https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors -O /notebooks/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors!wget https://huggingface.co/AICademyHK/SDXL_MODELS/resolve/main/ESRGAN/4x_foolhardy_Remacri.pth?download=true -O /notebooks/stable-diffusion-webui/models/ESRGAN/4x_foolhardy_Remacri.pth!wget https://huggingface.co/AICademyHK/SDXL_MODELS/resolve/main/Retro_Illustration.safetensors?download=true -O /notebooks/stable-diffusion-webui/Lora/Retro_Illustration.safetensors!wget https://huggingface.co/AICademyHK/SDXL_MODELS/resolve/main/Southern-Spring-sdxl_v1.safetensors?download=true -O /notebooks/stable-diffusion-webui/models/Lora/Southern-Spring-sdxl_v1.safetensors!wget https://huggingface.co/AICademyHK/SDXL_MODELS/resolve/main/csal_scenery_XL.safetensors?download=true -O /notebooks/stable-diffusion-webui/models/Lora/csal_scenery_XL.safetensors!wget https://huggingface.co/AICademyHK/SDXL_MODELS/resolve/main/landscape-painting-sdxl_v2.safetensors?download=true -O /notebooks/stable-diffusion-webui/models/Lora/landscape-painting-sdxl_v2.safetensors!wget https://huggingface.co/AICademyHK/SDXL_MODELS/resolve/main/traditional_watercolor_painting.safetensors?download=true -O /notebooks/stable-diffusion-webui/models/Lora/traditional_watercolor_painting.safetensors!wget https://huggingface.co/AICademyHK/SDXL_MODELS/resolve/main/%E5%9B%BD%E9%A3%8E%E6%8F%92%E7%94%BBSDXL.safetensors?download=true -O /notebooks/stable-diffusion-webui/models/Lora/CHIN_ILLU_SDXL.safetensors!wget https://huggingface.co/AICademyHK/SDXL_MODELS/resolve/main/psyAnimatedXL_v11.safetensors?download=true -O /notebooks/stable-diffusion-webui/models/Stable-diffusion/psyAnimatedXL_v11.safetensors

## Use the Stable Diffusion Web UI

Run the cell below to setup, and launch the Automatic1111 Stable Diffusion Web UI from this notebook. Click the Gradio link after the setup completes to access the Web UI in your local browser. 

Skip this step to use the Diffusers library methodology.

### Loading in the Stable Diffusion models for the Web UI 

To make accessing the Stable Diffusion models easy and not take up any storage, we have added the Stable Diffusion v1-5 and v2 models as mountable public datasets!

> These files will not count towards the storage limit for free notebooks.

To access the model files:

First, navigate to the "Data Sources" tab using the navigator on the far left of the page.

Next, click "Public" to switch into the Gradient Public Datasets, and scroll down until you find the Stable Diffusion datasets near the bottom of the list. You can mount `stable-diffusion-classic` for access to the v1-5 model checkpoints for the web UI, `stable-diffusion-diffusers` for the v1-5 models as Diffusers StableDiffusionPipeline binaries, `stable-diffusion-classic-v2` for access to the v2 model checkpoints and config file for use with the web UI, and `stable-diffusion-diffusers-v2` for access to the v2 models as Diffusers StableDiffusionPipeline binaries.

Finally, click "mount" to make any of these Public Datasets accessible from the "datasets" directory. This directory is in the root folder, so access it with the path `~/../datasets/stable-diffusion-classic/SDv1.5.ckpt`. For example, after mounting, the v1-5 checkpoint can be found at `~/../datasets/stable-diffusion-classic/SDv1.5.ckpt`

In [ ]:
# Launch the Stable Diffusion Web UI. 
# Be sure to hash out whichever you are not using.
%cd /notebooks/stable-diffusion-webui

## Launch Web UI for Stable Diffusion v1-x models.
!python launch.py --share


In [ ]:

#%cd /notebooks/stable-diffusion-webui/extensions/sd-webui-controlnet/models
#!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth
#!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth
#!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth
#!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth
#!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth
#!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth
#!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth
#!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth
#!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth
#!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth
#%cd /notebooks